# core

> Once upon a midnight dreary

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

## Meet Elements

In [ ]:
#| export
from molmass.elements import ELEMENTS

In [ ]:
#| export
class HelloElement:
    "Meet the `element`!"
   
    def __init__(self, element: str): self.element = ELEMENTS[element]
        
    def fun_fact(self):
        "Fun fact about me."
        return self.element.description

In [ ]:
#| hide
test_eq(HelloElement('Li').fun_fact().split()[-1], 'psychomedicine.')

In [ ]:
HelloElement('Li').fun_fact()

'Socket silvery metal. First member of group 1 of the periodic table. Lithium salts are used in psychomedicine.'

## Calculate Relative Mass Errors

In [ ]:
#| export
def mass_error_ppm(theor: float, obsvd: float):
    """Calculate relative mass error."""
    return round(1e6*(obsvd/theor - 1), 2)

In [ ]:
#| hide
test_eq(mass_error_ppm(120.0000, 120.0120), 100.0)
test_eq(mass_error_ppm(123.1234, 123.1230), -3.25)

In [ ]:
mass_error_ppm(123.1234, 123.1230)

-3.25

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

## Plot some plots

In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.sampledata.periodic_table import elements
from bokeh.transform import dodge, factor_cmap
output_notebook()

Loading BokehJS ...

### Interactive Bokeh

In [ ]:
periods = ["I", "II", "III", "IV", "V", "VI", "VII"]
groups = [str(x) for x in range(1, 19)]

df = elements.copy()
df["atomic mass"] = df["atomic mass"].astype(str)
df["group"] = df["group"].astype(str)
df["period"] = [periods[x-1] for x in df.period]
df = df[df.group != "-"]
df = df[df.symbol != "Lr"]
df = df[df.symbol != "Lu"]

cmap = {
    "alkali metal"         : "#a6cee3",
    "alkaline earth metal" : "#1f78b4",
    "metal"                : "#d93b43",
    "halogen"              : "#999d9a",
    "metalloid"            : "#e08d49",
    "noble gas"            : "#eaeaea",
    "nonmetal"             : "#f1d4Af",
    "transition metal"     : "#599d7A",
}

TOOLTIPS = [
    ("Name", "@name"),
    ("Atomic number", "@{atomic number}"),
    ("Atomic mass", "@{atomic mass}"),
    ("Type", "@metal"),
    ("CPK color", "$color[hex, swatch]:CPK"),
    ("Electronic configuration", "@{electronic configuration}"),
]

p = figure(title="Periodic Table (omitting LA and AC Series)", width=1000, height=450,
           x_range=groups, y_range=list(reversed(periods)),
           tools="hover", toolbar_location=None, tooltips=TOOLTIPS)

r = p.rect("group", "period", 0.95, 0.95, source=df, fill_alpha=0.6, legend_field="metal",
           color=factor_cmap('metal', palette=list(cmap.values()), factors=list(cmap.keys())))

text_props = dict(source=df, text_align="left", text_baseline="middle")

x = dodge("group", -0.4, range=p.x_range)

p.text(x=x, y="period", text="symbol", text_font_style="bold", **text_props)

p.text(x=x, y=dodge("period", 0.3, range=p.y_range), text="atomic number",
       text_font_size="11px", **text_props)

p.text(x=x, y=dodge("period", -0.35, range=p.y_range), text="name",
       text_font_size="7px", **text_props)

p.text(x=x, y=dodge("period", -0.2, range=p.y_range), text="atomic mass",
       text_font_size="7px", **text_props)

p.text(x=["3", "3"], y=["VI", "VII"], text=["LA", "AC"], text_align="center", text_baseline="middle")

p.outline_line_color = None
p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_standoff = 0
p.legend.orientation = "horizontal"
p.legend.location ="top_center"
p.hover.renderers = [r] # only hover element boxes

show(p)

### Static Altair

In [ ]:
import altair as alt
import panel as pn
import param

In [ ]:
pn.extension('vega')

In [ ]:
elements.columns

Index(['atomic number', 'symbol', 'name', 'atomic mass', 'CPK',
       'electronic configuration', 'electronegativity', 'atomic radius',
       'ion radius', 'van der Waals radius', 'IE-1', 'EA', 'standard state',
       'bonding type', 'melting point', 'boiling point', 'density', 'metal',
       'year discovered', 'group', 'period'],
      dtype='object')

In [ ]:
period = 4
certain_elements = elements.query(f"period == {period}")
alt.Chart(certain_elements, title=f"period {period} elements").mark_point().encode(
    x=alt.X('symbol:N', sort=certain_elements.symbol.to_numpy()),
    y='electronegativity',
)

alt.Chart(...)

In [ ]:
class PeriodSelector(param.Parameterized):
    period = param.Integer(6, bounds=(1, 6))
    
    @param.depends('period', watch=True)
    def _update_data(self):
        return elements.query(f"period == {self.period}").copy()
    
    def view(self):
        data = self._update_data()
        chart = (
            alt.Chart(data, title=f"period {self.period} elements")
            .mark_point()
            .encode(
                x=alt.X('symbol:N', sort=data.symbol.to_numpy()),
                y='electronegativity',
                tooltip=['symbol','atomic mass']
            )
            .properties(width=400, height=200)
        )
        return pn.Pane(chart)

period_selector = PeriodSelector(name='Periodic Trends in Electronegativity')
pn.Column(period_selector.param, period_selector.view)

Column
    [0] Column(margin=5, name='Periodic Trends i..., width=300)
        [0] StaticText(value='<b>Periodic T...)
        [1] IntSlider(end=6, name='Period', start=1, value=6)
    [1] ParamMethod(method)